In [2]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np

from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    log_loss,
    confusion_matrix
)

import xgboost as xgb
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

import seaborn as sns
from sklearn import model_selection

## Data import

In [3]:
df = pd.read_csv("credit_risk_dataset_red_ml_ord.csv")

# scaled
#df = pd.read_csv("credit_risk_dataset_red_ml_ord_scaled.csv")

In [4]:
# Train/test set
label = df['loan_status'] # labels
features = df.drop('loan_status',axis=1) # features
x_train, x_test, y_train, y_test = model_selection.train_test_split(features, label, 
                                                                    random_state=42, test_size=.30)

x_all = features
y_all = label

## Auxiliary functions

In [5]:
def create_prediction_df(model, X, y):
    """
    Creates a DataFrame containing the actual labels, model predictions, 
    and the confidence (probability) of the predicted class.

    Parameters:
    - model: A trained estimator (must have .predict(), ideally .predict_proba())
    - X: Input features (DataFrame or array-like)
    - y: True labels (Series or array-like)

    Returns:
    - pd.DataFrame with columns: ['y_true', 'y_pred', 'confidence']
    """
    
    # 1. Generate Predictions
    predictions = model.predict(X)
    
    # 2. Initialize the DataFrame
    # We use valid indices from y if it's a pandas object, otherwise default
    index = y.index if hasattr(y, 'index') else None
    results_df = pd.DataFrame({
        'y_true': y,
        'y_pred': predictions
    }, index=index)
    
    # 3. Calculate Confidence (if supported)
    if hasattr(model, "predict_proba"):
        # Get probabilities for all classes
        probas = model.predict_proba(X)
        
        # Extract the maximum probability for each row (the confidence of the chosen class)
        # axis=1 finds the max value across columns for each row
        confidence = np.max(probas, axis=1)
        
        results_df['confidence'] = confidence
    else:
        # Fallback for models that don't support probabilities (e.g., SVM without probability=True)
        results_df['confidence'] = np.nan
        print("Warning: Model does not support 'predict_proba'. Confidence set to NaN.")

    return results_df

In [6]:
def evaluate_classification(model, X, y):
    """
    Calculates accuracy, F1, precision, recall, support, and confusion matrix.

    Parameters:
    - model: The trained classification model (must have a .predict() method).
    - X: The features to predict on.
    - y: The true labels.

    Returns:
    - metrics: A dictionary containing 'accuracy' and 'confusion_matrix'.
    - report_df: A pandas DataFrame containing precision, recall, f1-score, and support per class.
    """
    # 1. Generate Predictions
    y_pred = model.predict(X)

    # 2. Calculate Metrics
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)

    # 3. Calculate Precision, Recall, F1, and Support
    # precise_recall_fscore_support returns lists for each class
    precision_l, recall_l, f1_l, support_l = precision_recall_fscore_support(y, y_pred)

    # 4. Create a DataFrame for per-class metrics
    # We attempt to use model.classes_ if available for row names, otherwise simple indices
    try:
        classes = model.classes_
    except AttributeError:
        classes = list(range(len(precision_l)))

    report_df = pd.DataFrame({
        'Precision': precision_l,
        'Recall': recall_l,
        'F1-Score': f1_l,
        'Support': support_l
    }, index=classes)

    # 5. Calculate Confusion Matrix
    conf_matrix = confusion_matrix(y, y_pred)

    
    y_proba = model.predict_proba(X)[:, 1]

    # 6. ROC AUC Score
    # Measures ability to distinguish between classes. 1.0 is perfect, 0.5 is random guessing.
    roc_auc = roc_auc_score(y, y_proba)

    # Bundle everything into a dictionary
    metrics = {
        "accuracy": accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        "confusion_matrix": conf_matrix,
        "roc_auc": roc_auc
    }


    print(report_df)

    for key in metrics.keys():
        print(key, metrics[key])
        
    print(conf_matrix)

    prediction_df = create_prediction_df(model, X, y)

    df_confidence = pd.concat([X, prediction_df['confidence']], axis=1)

    return metrics, report_df, prediction_df, df_confidence

In [7]:
def get_confidence_stats(df):
    """
    Returns a summary table with mean, min, and max confidence 
    for correct vs. incorrect predictions.
    """
    # Create a copy to avoid modifying the original dataframe
    df_stats = df.copy()
    
    # Create a boolean column for grouping
    df_stats['prediction_status'] = df_stats.apply(
        lambda row: 'Correct' if row['y_true'] == row['y_pred'] else 'Incorrect', 
        axis=1
    )
    
    # Group by the status and calculate statistics on the 'confidence' column
    summary = df_stats.groupby('prediction_status')['confidence'].agg(['mean', 'std', 'min', 'max', 'count'])
    
    return summary

In [8]:
def lowest_x_confidence(df, x):

    return df.nsmallest(x, 'confidence')

## XGBoost train

In [9]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42) 

xgb_model.fit(x_train, y_train)

# Substitute as you want x_train, y_train; x_test, y_test; x_all y_all
metrics, report_df, prediction_df, df_confidence = evaluate_classification(xgb_model, x_all, y_all)


   Precision    Recall  F1-Score  Support
0   0.976131  0.993606  0.984791     2346
1   0.975490  0.912844  0.943128      654
accuracy 0.976
Precision 0.9754901960784313
Recall 0.9128440366972477
F1-Score 0.943127962085308
confusion_matrix [[2331   15]
 [  57  597]]
roc_auc 0.9801829387518869
[[2331   15]
 [  57  597]]


In [ ]:
selected_features = [
    "person_home_ownership_RENT",
    "loan_grade",
    "loan_percent_income",
    "person_home_ownership_OWN",
    "person_home_ownership_MORTGAGE",
    "loan_intent_DEBTCONSOLIDATION",
    "loan_int_rate",
    "loan_intent_HOMEIMPROVEMENT",
    "loan_intent_MEDICAL",
    "loan_amnt",
    "loan_status"
]

update_df = df[selected_features]
df_confidence_prova = pd.concat([update_df, prediction_df['confidence']], axis=1)
print(len(df_confidence_prova))

3000


In [29]:
number_of_points_from_which_to_balance = 300

lowest = lowest_x_confidence(df_confidence_prova, number_of_points_from_which_to_balance)
# THIS IS UNBALANCED 

counts = lowest["loan_status"].value_counts()
n_min = counts.min()

lowest_balanced = ( 
    pd.concat([
        lowest[lowest["loan_status"] == 0 ].sample(n = n_min, random_state=42), 
        lowest[lowest["loan_status"] == 1 ].sample(n = n_min, random_state=42), 
        ])
        .sample(frac=1, random_state = 42)
        .reset_index(drop=True)
)
lowest_balanced["loan_status"].value_counts()


loan_status
0    142
1    142
Name: count, dtype: int64

In [30]:
lowest_balanced.to_csv('name_of_file.csv', index=False)

In [31]:
X_paolo = lowest_balanced.drop(columns=['loan_status', 'confidence']).values
y_paolo = lowest_balanced['loan_status'].values

print(lowest_balanced['loan_status'].value_counts(normalize=True))

from sklearn.model_selection import train_test_split


# Split by percentage
train_percentage = 0.8  # 80% train, 20% test
X_train, X_test, train_labels, test_labels = train_test_split(
    X_paolo, y_paolo, train_size=train_percentage, random_state=42, shuffle=True
)


import xgboost
xgb_model = xgboost.XGBClassifier(objective = "binary:logistic", random_state=42)

xgb_model.fit(X_train, train_labels)

y_pred = xgb_model.predict(X_test)

loan_status
0    0.5
1    0.5
Name: proportion, dtype: float64


In [32]:

from sklearn.metrics import precision_score, recall_score, f1_score

print("Precision:", precision_score(test_labels, y_pred))
print("Recall:", recall_score(test_labels, y_pred))
print("F1:", f1_score(test_labels, y_pred))

Precision: 0.53125
Recall: 0.6071428571428571
F1: 0.5666666666666667
